In [1]:
import os
from pymilvus import MilvusClient

In [2]:
client = MilvusClient('../data/task_embeddings.db')
client.load_collection('swe_bench_verified')
client.load_collection('swe_bench_lite')

In [4]:
client.list_indexes(collection_name='swe_bench_lite')

['vector_index']

In [5]:
client.describe_index('swe_bench_verified', 'vector_index')

{'index_type': 'FLAT',
 'metric_type': 'COSINE',
 'dim': '768',
 'field_name': 'vector',
 'index_name': 'vector_index',
 'total_rows': 0,
 'indexed_rows': 0,
 'pending_index_rows': 0,
 'state': 'Finished'}

In [6]:
client.describe_index('swe_bench_lite', 'vector_index')

{'index_type': 'FLAT',
 'metric_type': 'COSINE',
 'dim': '768',
 'field_name': 'vector',
 'index_name': 'vector_index',
 'total_rows': 0,
 'indexed_rows': 0,
 'pending_index_rows': 0,
 'state': 'Finished'}

In [7]:
swe_bench_lite = client.get(collection_name='swe_bench_lite', ids=[idx for idx in range(0, 53)])

In [8]:
swe_bench_verified = client.get(collection_name='swe_bench_verified', ids=[idx for idx in range(0, 500)])

In [9]:
swe_bench_lite_queries = [data['vector'] for data in swe_bench_lite]
swe_bench_verified_queries = [data['vector'] for data in swe_bench_lite]

# Qualitative analysis

In [11]:
results = client.search(collection_name='swe_bench_verified', data=[swe_bench_lite[0]['vector']], limit=2000, search_params={"metric_type": "COSINE"})

In [12]:
results[0][0]

{'id': 130, 'distance': 0.9086493253707886, 'entity': {}}

Most similar result from the verified dataset to the first problem statement in the lite dataset.

In [13]:
print(client.get(collection_name='swe_bench_verified', ids=[results[0][0]['id']])[0]['problem_statement'])

MiddlewareNotUsed leaves undesired side effects when loading middleware in ASGI context
Description
	
I experienced strange issues when working with ​ASGI , ​django-debug-toolbar and my own small middleware. It was hard problem to debug, I uploaded an example project here: ​https://github.com/hbielenia/asgi-djangotoolbar-bug (the name is misleading - I initially thought it's a bug with django-debug-toolbar).
The SESSION_FILE_PATH setting is intentionally broken to cause a 500 error. When starting the application and accessing /admin (any location really, but I wanted to leave it at a minimum and didn't add any views) it gives TypeError: object HttpResponse can't be used in 'await' expression. Commenting out asgi_djangotoolbar_bug.middleware.DummyMiddleware fixes the issue (in that I receive a 500 ImproperlyConfigured exception). I'm not sure about the overall role of django-debug-toolbar here - removing it causes Daphne to return a 500 error page but without debug information and there

The first problem statement in the lite dataset.

In [14]:
print(client.get(collection_name='swe_bench_lite', ids=[0])[0]['problem_statement'])

sqlmigrate wraps it's outpout in BEGIN/COMMIT even if the database doesn't support transactional DDL
Description
	 
		(last modified by Simon Charette)
	 
The migration executor only adds the outer BEGIN/COMMIT ​if the migration is atomic and ​the schema editor can rollback DDL but the current sqlmigrate logic only takes migration.atomic into consideration.
The issue can be addressed by
Changing sqlmigrate ​assignment of self.output_transaction to consider connection.features.can_rollback_ddl as well.
Adding a test in tests/migrations/test_commands.py based on ​an existing test for non-atomic migrations that mocks connection.features.can_rollback_ddl to False instead of overdidding MIGRATION_MODULES to point to a non-atomic migration.
I marked the ticket as easy picking because I included the above guidelines but feel free to uncheck it if you deem it inappropriate.



Most dissimilar result from the verified dataset to the first problem statement in the lite dataset.

In [16]:
print(client.get(collection_name='swe_bench_verified', ids=[results[0][-1]['id']])[0])

{'id': 16, 'instance_id': 'astropy__astropy-7166', 'problem_statement': "InheritDocstrings metaclass doesn't work for properties\nInside the InheritDocstrings metaclass it uses `inspect.isfunction` which returns `False` for properties.\n", 'trajectory': '', 'vector': [-0.062099896, 0.08621872, -0.11391117, -0.004229007, -0.116731524, -0.0054171667, -0.011990221, -0.0206835, 0.105047606, -0.053270746, -0.15306564, 0.01190524, -0.006012328, 0.026266038, 0.08489973, -0.14850765, 0.00020400897, 0.0839852, 0.09256988, -0.029696435, 0.243818, -0.079614125, 0.016626861, 0.012063121, -0.10178837, -0.17911328, -0.013824479, -0.059303645, 0.022514997, -0.012777827, -0.11344772, -0.18890014, 0.0012096227, -0.15699914, 0.08238656, -0.12212808, 0.031483613, -0.11565126, 0.15224515, -0.089157104, 0.017278733, -0.002086968, -0.009951046, 0.08554277, 0.057850476, -0.080476515, 0.08071871, -0.10240512, 0.12175029, 0.036159366, -0.024516666, 0.023758953, -0.02231913, -0.009612218, 0.13675852, -0.1557475

## Searching with a short problem statement as query

In [18]:
results_short_verified_to_lite = client.search(collection_name='swe_bench_lite', data=[swe_bench_verified[16]['vector']], limit=2000)

In [21]:
print(client.get(collection_name='swe_bench_lite', ids=[results_short_verified_to_lite[0][0]['id']])[0]['problem_statement'])

expr.atoms() should return objects with no args instead of subclasses of Atom
`expr.atoms()` with no arguments returns subclasses of `Atom` in `expr`. But the correct definition of a leaf node should be that it has no `.args`. 

This should be easy to fix, but one needs to check that this doesn't affect the performance. 




In [22]:
print(client.get(collection_name='swe_bench_lite', ids=[results_short_verified_to_lite[0][-1]['id']])[0]['problem_statement'])

Settings are cleaned insufficiently.
Description
	
Posting publicly after checking with the rest of the security team.
I just ran into a case where django.views.debug.SafeExceptionReporterFilter.get_safe_settings() would return several un-cleansed values. Looking at cleanse_setting() I realized that we ​only take care of `dict`s but don't take other types of iterables into account but ​return them as-is.
Example:
In my settings.py I have this:
MY_SETTING = {
	"foo": "value",
	"secret": "value",
	"token": "value",
	"something": [
		{"foo": "value"},
		{"secret": "value"},
		{"token": "value"},
	],
	"else": [
		[
			{"foo": "value"},
			{"secret": "value"},
			{"token": "value"},
		],
		[
			{"foo": "value"},
			{"secret": "value"},
			{"token": "value"},
		],
	]
}
On Django 3.0 and below:
>>> import pprint
>>> from django.views.debug import get_safe_settings
>>> pprint.pprint(get_safe_settings()["MY_SETTING"])
{'else': [[{'foo': 'value'}, {'secret': 'value'}, {'token': 'value'}],
		 [{'

It seems like shorter problem statements are matched with shorter ones and longer problem statements with longer ones. How similar the actual trajectories will be is up to debate.

## Similarity within SWE-bench Verified

Perhaps a later dataset baseline will lead to better actual similarity, let's briefly explore this.

In [23]:
results_within_verified = client.search(collection_name='swe_bench_verified', data=[swe_bench_verified[2]['vector']], limit=2000)

In [30]:
output = client.get(collection_name='swe_bench_verified', ids=[2])[0]
print(output['instance_id'])
print(output['problem_statement'])

astropy__astropy-13236
Consider removing auto-transform of structured column into NdarrayMixin
<!-- This comments are hidden when you submit the issue,
so you do not need to remove them! -->

<!-- Please be sure to check out our contributing guidelines,
https://github.com/astropy/astropy/blob/main/CONTRIBUTING.md .
Please be sure to check out our code of conduct,
https://github.com/astropy/astropy/blob/main/CODE_OF_CONDUCT.md . -->

<!-- Please have a search on our GitHub repository to see if a similar
issue has already been posted.
If a similar issue is closed, have a quick look to see if you are satisfied
by the resolution.
If not please go ahead and open an issue! -->

### Description
<!-- Provide a general description of the feature you would like. -->
<!-- If you want to, you can suggest a draft design or API. -->
<!-- This way we have a deeper discussion on the feature. -->

Currently if you add a structured `np.array` to a Table, it gets turned into an `NdarrayMixin` (via the co

In [32]:
output = client.get(collection_name='swe_bench_verified', ids=[results_short_verified_to_lite[0][0]['id']])[0]
print(output['instance_id'])
print(output['problem_statement'])

django__django-11451
ModelBackend.authenticate() shouldn't make a database query when username is None
Description
	
It's easier to explain my issue by adding a comment in the current implementation of ModelBackend.authenticate():
	def authenticate(self, request, username=None, password=None, **kwargs):
		if username is None:
			username = kwargs.get(UserModel.USERNAME_FIELD)
		# At this point, username and password can be None,
		# typically if credentials are provided for another backend.
		# Continuing makes a useless database query and runs
		# the password hasher needlessly (which is expensive).
		try:
			user = UserModel._default_manager.get_by_natural_key(username)
		except UserModel.DoesNotExist:
			# Run the default password hasher once to reduce the timing
			# difference between an existing and a nonexistent user (#20760).
			UserModel().set_password(password)
		else:
			...
My suggestion is to shortcut with:
		if username is None or password is None:
			return
I noticed thi

In [36]:
output = client.get(collection_name='swe_bench_verified', ids=[results_short_verified_to_lite[0][-1]['id']])[0]
print(output['instance_id'])
print(output['problem_statement'])

astropy__astropy-14096
Subclassed SkyCoord gives misleading attribute access message
I'm trying to subclass `SkyCoord`, and add some custom properties. This all seems to be working fine, but when I have a custom property (`prop` below) that tries to access a non-existent attribute (`random_attr`) below, the error message is misleading because it says `prop` doesn't exist, where it should say `random_attr` doesn't exist.

```python
import astropy.coordinates as coord


class custom_coord(coord.SkyCoord):
    @property
    def prop(self):
        return self.random_attr


c = custom_coord('00h42m30s', '+41d12m00s', frame='icrs')
c.prop
```

raises
```
Traceback (most recent call last):
  File "test.py", line 11, in <module>
    c.prop
  File "/Users/dstansby/miniconda3/lib/python3.7/site-packages/astropy/coordinates/sky_coordinate.py", line 600, in __getattr__
    .format(self.__class__.__name__, attr))
AttributeError: 'custom_coord' object has no attribute 'prop'
```

